In [1]:
import os

# 设置环境变量
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

# 打印环境变量以确认设置成功
print(os.environ.get('HF_ENDPOINT'))

https://hf-mirror.com


In [2]:
from transformers import AutoTokenizer, AutoModel
from tokenizers import Tokenizer
from transformers import GPT2LMHeadModel, AutoConfig,GPT2Tokenizer
from transformers import AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding

2024-10-18 19:37:08.143082: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-18 19:37:08.155839: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-18 19:37:08.170783: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-18 19:37:08.175307: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-18 19:37:08.186905: I tensorflow/core/platform/cpu_feature_guar

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained("dnagpt/dna_eng_bpe")
tokenizer.pad_token = tokenizer.eos_token

In [4]:
model = AutoModelForSequenceClassification.from_pretrained('gpt_dna_eng_v1', num_labels=2)
model.config.pad_token_id = model.config.eos_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt_dna_eng_v1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
model

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50000, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)

In [6]:
from datasets import load_dataset
# load ~11k samples from promoters prediction dataset
dataset = load_dataset("dnagpt/dna_promoters")['train'].train_test_split(test_size=0.1)

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sequence', 'promoter_presence'],
        num_rows: 10656
    })
    test: Dataset({
        features: ['sequence', 'promoter_presence'],
        num_rows: 1184
    })
})

In [8]:
dataset['train'][0]

{'sequence': 'GGGCTCAGAGACTGGGTTTGTTTTGGGGAGTCTGCACCTATTTGCTGAGTGAATGTATGTGTGTGTGCATTTGAGAGCACACCTCTGTATGATTCGGGTGTGAGTGTGTGTGAGGAAACGTGGGCAGGCGAGGAGTGTTTGGGAGCCAGGTGCAGCTGGGGTGTGAGTGTGTAAGCAAGCAGCTATGAGGCTGGGCATTGCTTCTCCTCCTCTTCTCCAGCTCCCAGCCTTTCTTCCCCGGGACTCCTGGGGCTCCAGGATGCCCCCAAGATCCCCTCCACAAGTGGATAATTTGGGCTGCCTTGTCTGACCCTCCAATTATACACTCCTTTGCCTCTTTCCGTCATTCCATAACCACCCAACCCCTACTCCACCGGGAGGGGGTTGGGCATACCTGGATCCACCCCGTTTCTGGGGGGTGAGCGGGGCTTGGCAGGGCTGCGCGGAGGGCGCGGGGGTGGGGCCCGGGGCGGAGCGGCCCGGGGCGGAGGGCGCGGGCTAGCTGAGACTACAGGCGTGCACCATCACGTCCGGCTAATTTTTTGTATTTTAGTAGAGAGGGGGTTTCACCATGTTGGCTAGGATGGTCTCGATCTCCTGCGCTCCACCAGCTTCAGGGGCGGCATGGGGTCCGGGGGCCTGGCCACCGGGATAGCCGGGGGTCTGGCAGGAATGGGAGGCATCCAGAACGAGAAGGAGATCCTCCACCTGGGGGAGAAGAGCATAATAACGTCATTTCCTGCCCTGAAAGCAGCCTCGAGGGCCAACAACACCTGCTGTCCGTGTCCATGCCCGGTTGGTTCCATCCGCGCACCTAGCCACAGGGTCCCTAAGAGCAGCAGCTAGGCATGGGAGGGCTCTTTCCCAGGAGAGAGGGGGAAGGGGACAGGGTTGAGAGCTCCGAGCCGTCCACCTGTGGCTCCGGCTTCCGAAGCGGCTCCGGGGCGGGGGCGGGGCCTCACTCTGCGATATAACTCGGGTCGCGC

In [9]:
print('tokens: ', ' '.join(tokenizer.tokenize(dataset['train'][0]['sequence'])))

tokens:  GGGC TCAGAG ACTGGG TTTGTTTT GGGG AGTC TGCACC TATTTGC TGAGTG AATG TATGTGTG TGTGC ATTTGAG AGCACACC TCTGTATG ATTC GGGTGTG AGTGTGTGTG AGG AAACG TGGGC AGGCG AGGAGTG TTTGGG AGCC AGGTGC AGCTGGGG TGTG AGTGTG TAAGC AAGC AGCTATG AGGCTGGGC ATTGC TTCTCC TCCTCTTC TCCAGC TCCCAGCC TTTC TTCCCC GGG ACTCC TGGGGC TCCAGG ATGCC CCCAAG ATCCCC TCC ACAAG TGGATAA TTTGGGC TGCC TTGTCTG ACCCTCC AATT ATACAC TCC TTTGCC TCTTTCC G TCATTCC ATAACC ACCC AACCCC TAC TCCACC GGG AGGGGG TTGGGC ATACC TGGATCC ACCCC GTTTC TGGGGGG TGAGC GGGGC TTGGC AGGGCTGC GCGG AGGGC GCGG GGG TGGGGCCC GGGGC GGAGC GGCCC GGGGC GG AGGGCGC GGGC TAGCTGAGAC TACAGGC GTGC ACCATC ACG TCCGGC TAATTTTTTGTATT TTAGTAG AGAGGGGG TTTCACCATGTTGGC TAGGATGG TCTCGATCTCC TGCGC TCCACC AGCTTC AGGGGC GGC ATGGGG TCCGG GGGCC TGGCC ACCGGG ATAGCC GGGGG TCTGGC AGG AATGGG AGGCATCC AGAACG AGAAGG AGATCC TCCACC TGGGGG AGAAG AGCATAA TAACG TCATT TCCTGCCC TGAA AGCAGCC TCG AGGGCC AACAAC ACCTGC TGTCC GTGTCC ATGCCC GG TTGGTTCC ATCC GCGC ACCTAGCC ACAGGG TCCCTAAG AGCAGC AGCTAG

In [10]:
print('# tokens: ', len(tokenizer.tokenize(dataset['train'][0]['sequence'])))

# tokens:  344


In [11]:
dataset = dataset.rename_column("promoter_presence", "label")
dataset

DatasetDict({
    train: Dataset({
        features: ['sequence', 'label'],
        num_rows: 10656
    })
    test: Dataset({
        features: ['sequence', 'label'],
        num_rows: 1184
    })
})

In [12]:
# 2. tokenize
def tokenize_function(examples):
    return tokenizer(examples['sequence'], truncation=True, padding='max_length', max_length=1024)

# 3. 对数据集应用分词函数
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# 4. 创建一个数据收集器，用于动态填充和遮蔽
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/10656 [00:00<?, ? examples/s]

Map:   0%|          | 0/1184 [00:00<?, ? examples/s]

In [13]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sequence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 10656
    })
    test: Dataset({
        features: ['sequence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1184
    })
})

In [14]:
from transformers import TrainingArguments, Trainer
import numpy as np



def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy': (predictions==labels).sum() / len(labels)}

# change training hyperparameters to archive better quality
training_args = TrainingArguments(
    output_dir="ds_job_dna_eng_1_run",
    learning_rate=1e-5,
    lr_scheduler_type="constant_with_warmup",
    warmup_ratio=0.1,
    optim='adamw_torch',
    weight_decay=0.0,
    per_device_train_batch_size=40,
    per_device_eval_batch_size=40,
    num_train_epochs=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/root/miniconda3/lib/python3.12/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Detected kernel version 4.19.90, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.639900,0.480039,0.774493
2,0.431900,0.447300,0.798142
3,0.316500,0.482207,0.809122
4,0.243400,0.375102,0.854730
5,0.178900,0.459049,0.829392
6,0.105000,0.525342,0.843750
7,0.073600,1.140752,0.798142
8,0.058400,0.850573,0.831926
9,0.053000,1.370116,0.825169
10,0.047200,1.416664,0.831081


TrainOutput(global_step=2670, training_loss=0.2147740356931079, metrics={'train_runtime': 4872.7282, 'train_samples_per_second': 21.869, 'train_steps_per_second': 0.548, 'total_flos': 5.568756348616704e+16, 'train_loss': 0.2147740356931079, 'epoch': 10.0})